In [ ]:
import os
import functools

import cv2
import numpy as np

from tests.metaworld.envs.mujoco.sawyer_xyz.test_scripted_policies import ALL_ENVS, test_cases_latest_nonoise


In [ ]:
def trajectory_generator(env, policy, act_noise_pct, res=(640, 480), camera='corner'):
    action_space_ptp = env.action_space.high - env.action_space.low

    env.reset()
    env.reset_model()
    o = env.reset()

    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        # Camera is one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
        yield r, done, info, env.sim.render(*res[::-1], mode='offscreen', camera_name=camera)[:,:,::-1]

def writer_for(tag, fps, res):
    if not os.path.exists('movies'):
        os.mkdir('movies')
    return cv2.VideoWriter(
        f'movies/{tag}.avi',
        cv2.VideoWriter_fourcc('M','J','P','G'),
        fps,
        res
    )

In [ ]:
resolution = (360, 540)
attempts = 3
camera = 'corner' # one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
flip=False # if True, flips output image 180 degrees

config = [
    # env, action noise pct, quit on success
    # ('assembly-v2', np.zeros(4), True),
    # ('basketball-v2', np.zeros(4), True),
    # ('bin-picking-v2', np.zeros(4), True),
    # ('box-close-v2', np.zeros(4), True),
    # ('button-press-topdown-v2', np.zeros(4), True),
    # ('button-press-topdown-wall-v2', np.zeros(4), True),
    # ('button-press-v2', np.zeros(4), True),
    # ('button-press-wall-v2', np.zeros(4), True),
    # ('coffee-button-v2', np.zeros(4), True),
    # ('coffee-pull-v2', np.zeros(4), True),
    # ('coffee-push-v2', np.zeros(4), True),
    # ('dial-turn-v2', np.zeros(4), True),
    # ('disassemble-v2', np.zeros(4), True),
    # ('door-close-v2', np.zeros(4), True),
    # ('door-lock-v2', np.zeros(4), True),
    # ('door-open-v2', np.zeros(4), True),
    # ('door-unlock-v2', np.zeros(4), True),
    # ('drawer-close-v2', np.zeros(4), True),
    # ('drawer-open-v2', np.zeros(4), True),
    # ('faucet-open-v2', np.zeros(4), True),
    # ('faucet-close-v2', np.zeros(4), True),
    # ('hammer-v2', np.zeros(4), True),
    # ('hand-insert-v2', np.zeros(4), True),
    # ('handle-press-side-v2', np.zeros(4), True),
    # ('handle-press-v2', np.zeros(4), True),
    # ('handle-pull-side-v2', np.zeros(4), True),
    # ('handle-pull-v2', np.zeros(4), True),
    # ('lever-pull-v2', np.zeros(4), True),
    # ('peg-insert-side-v2', np.zeros(4), True),
    # ('peg-unplug-side-v2', np.zeros(4), True),
    # ('pick-out-of-hole-v2', np.zeros(4), True),
    ('pick-place-v2', np.zeros(4), True),
    # ('pick-place-wall-v2', np.zeros(4), True),
    # ('plate-slide-v2', np.zeros(4), True),
    # ('plate-slide-side-v2', np.zeros(4), True),
    # ('plate-slide-back-v2', np.zeros(4), True),
    # ('plate-slide-back-side-v2', np.zeros(4), True),
    # ('push-back-v2', np.zeros(4), True),
    # ('push-v2', np.zeros(4), True),
    # ('push-wall-v2', np.zeros(4), True),
    # ('reach-v2', np.zeros(4), True),
    # ('reach-wall-v2', np.zeros(4), True),
    # ('shelf-place-v2', np.zeros(4), True),
    # ('soccer-v2', np.zeros(4), True),
    # ('stick-push-v2', np.zeros(4), True),
    # ('stick-pull-v2', np.zeros(4), True),
    # ('sweep-into-v2', np.zeros(4), True),
    # ('sweep-v2', np.zeros(4), True),
    # ('window-open-v2', np.zeros(4), True),
    # ('window-close-v2', np.zeros(4), True),
]

entries = (1, 1)
assert(entries[0] * entries[1] == len(config))
pixels_x = (resolution[1] + 40) * entries[1]
pixels_y = (resolution[0] + 200) * entries[0]

grid = np.full((500, pixels_y, pixels_x, 3), fill_value=255, dtype='uint8')
fps = 0

for i, (env, noise, quit_on_success) in enumerate(config):
    name = env[:-3]

    policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
    env = ALL_ENVS[env]()
    env._partially_observable = False
    env._freeze_rand_vec = False
    env._set_task_called = True

    fps = env.metadata['video.frames_per_second']
    x = (resolution[1] + 40) * (i % entries[1])
    y = (resolution[0] + 200) * (i // entries[1])
    i += 1
    cropped = grid[:, y:y + resolution[0], x:x + resolution[1]]

    for _ in range(attempts):
        j = 0
        info = { 'success': False }
        img = np.zeros_like(cropped)

        for r, done, info, img in trajectory_generator(env, policy, noise, resolution, camera):
            if flip: img = cv2.rotate(img, cv2.ROTATE_180)

            cropped[j] = img
            j += 1

            if quit_on_success and info['success']:
                break

        if info['success']:
            img[:, :, 1] = np.clip(img[:, :, 1] + 20, a_min=0, a_max=255)
            cropped[j:] = img
            break

    for j in range(500):
        o = 0
        # if len(name) > 22:
        #     o = 60

        cv2.putText(
            grid[j], name, (x + 20, y + resolution[0] + o + 70),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=1.3,
            color=(0,0,0),
            thickness=5
        )


In [ ]:
writer = writer_for('all_env_grid_2', fps, (pixels_x, pixels_y))
for i in range(grid.shape[0]):
    writer.write(grid[i])
writer.release()